In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import pickle
import cv2
import face_recognition
from tqdm.auto import tqdm
tqdm.pandas()

/home/l_sawaniewski/Dokumenty/DataWorkshop/DW_Olsztyn/repos/dw-olsztyn-project/dw_face_recognition_env/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def fetch_site(site_url):
    time.sleep(np.random.randint(1,5))
    headers = requests.utils.default_headers()
    headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',})
    html_file = requests.get(site_url, headers=headers)
    
    return BeautifulSoup(html_file.content, "lxml")

<img align="left" src="https://i.ytimg.com/vi/MKKuRMynthI/hqdefault.jpg" width="300">

In [3]:
# one list to rule them all
main_list = []

## SEJM

In [4]:
# only currently active
source_urls = [
    "https://www.sejm.gov.pl/Sejm9.nsf/poslowie.xsp?type=A"
]

In [5]:
for url in tqdm(source_urls):
    soup = fetch_site(url)
    
    # deputies displayed on page as a list of dicts
    deputies = [
        {
            "url":f"https://www.sejm.gov.pl{li.find('a')['href']}" 
        } 
        for ul in soup.find_all("ul", {"class":"deputies"}) 
        for li in ul.find_all("li")
    ]
    
    # updating deputies dictionaries with new details
    for deputy in tqdm(deputies):
        soup = fetch_site(deputy["url"])
        deputy["name"] = soup.find("h1").text
        try:
            deputy["photo_url"] = soup.find("div", {"id":"contentBody"}).find("img")["src"]
        except Exception as e:
            print(f"[WARNING] No photo found @: {deputy['url']} ({e})")
        try:
            deputy.update(
                zip(
                    ["elected_date", "list", "district", "votes", "political_presence", "club"], 
                    [
                        li.find_all("p")[1].text
                        for li in soup.find("div", {"class":"partia"}).find_all("li")
                    ]
                )
            )
        except Exception as e:
            print(f"[WARNING] No details(1) found @: {deputy['url']} ({e})")
        try:
            deputy.update(
                zip(
                    ["birth", "education", "graduated_school", "occupation"], 
                    [
                        li.find_all("p")[1].text
                        for li in soup.find("div", {"class":"cv"}).find_all("li") 
                        if "Tytuł" not in li.find_all("p")[0].text
                    ]
                )
            )
        except Exception as e:
            print(f"[WARNING] No details(2) found @: {deputy['url']} ({e})")
    
    # adding to main list of persons
    main_list.extend(deputies)

[WARNING] No photo found @: https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id=029&type=A ('NoneType' object has no attribute 'find')
[WARNING] No details(1) found @: https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id=029&type=A ('NoneType' object has no attribute 'find_all')
[WARNING] No details(2) found @: https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id=029&type=A ('NoneType' object has no attribute 'find_all')




## SENAT

In [6]:
source_urls = [
    "https://www.senat.gov.pl/sklad/senatorowie/"
]

In [7]:
for url in tqdm(source_urls):
    soup = fetch_site(url)
    
    # deputies displayed on page as a list of dicts
    deputies = [
        {
            "url":f"https://www.senat.gov.pl{div.find('a')['href']}" 
        } 
        for div in soup.find_all("div", {"class":"senator-kontener"}) 
    ]
    
    # updating deputies dictionaries with new details
    for deputy in tqdm(deputies):
        soup = fetch_site(deputy["url"])
        deputy["name"] = soup.find("div", {"class":"informacje"}).find('h2').text
        try:
            deputy["photo_url"] = "https://www.senat.gov.pl" + soup.find('div', {'class':'zdjecie'}).find('img')['src'] 
        except Exception as e:
            print(f"[WARNING] No photo found @: {deputy['url']} ({e})")
        try:
            # some of them have missing list information
            if len( soup.find("div", {"class":"informacje"}).find_all("li") ) == 4:
                deputy.update(
                    zip(
                        ["district", "political_presence"], 
                        [li.text for li in soup.find("div", {"class":"informacje"}).find_all("li")[:2]]
                    )
                )
            else:
                deputy.update(
                    zip(
                        ["district", "list", "political_presence"], 
                        [li.text for li in soup.find("div", {"class":"informacje"}).find_all("li")[:3]]
                    )
                )
        except Exception as e:
            print(f"[WARNING] No details(1) found @: {deputy['url']} ({e})")
        try:
            deputy["club"] = soup.find("div", {"class":"kluby"}).text.strip() 
        except:
            print(f"[WARNING] No club found @: {deputy['url']} ({e})")
        
    # adding to main list of persons
    main_list.extend(deputies)

## PREZYDENT

In [8]:
main_list.append(
    {
        "url":"https://www.prezydent.pl/prezydent/biografia-andrzeja-dudy/",
        "name":"Andrzej Duda", 
        "photo_url":"https://www.prezydent.pl/gfx/prezydent/pl/defaultopisy/5437/1/1/s697870047.jpg",
        "birth":"16-05-1972, Kraków", 
        "education":"wyższe",
        "occupation":"prezydent" # ;)
    }
)

## WYBORY 2020

In [9]:
main_list.extend([
    # Szymon Hołownia
    {
            "url":"https://pl.wikipedia.org/wiki/Szymon_Ho%C5%82ownia",
            "name":"Szymon Hołownia", 
            "photo_url":"https://cdn.galleries.smcloud.net/t/galleries/gf-42iV-uQSW-8dwR_szymon-holownia-664x442-nocrop.png",
            "birth":"13-04-1976, Rymanów", 
            "education":"wyższe",
            "occupation":"dziennikarz"
    },
    # Robert Biedroń
    {
            "url":"https://pl.wikipedia.org/wiki/Robert_Biedro%C5%84",
            "name":"Robert Biedroń", 
            "photo_url":"https://i.iplsc.com/robert-biedron/0009QBOR5YGDGA05-C123-F4.jpg",
            "birth":"03-09-1976, Rymanów", 
            "education":"wyższe"
    }
])

In [10]:
df = pd.DataFrame(main_list)

In [11]:
df.head()

,url,name,photo_url,elected_date,list,district,votes,political_presence,club,birth,education,graduated_school,occupation
0,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Andrzej Adamczyk,http://orka.sejm.gov.pl/Poslowie9.nsf/0/8A5101...,13-10-2019,Prawo i Sprawiedliwość,13 Kraków,29686,"poseł V kadencji, poseł VI kadencji, poseł VII...",Klub Parlamentarny Prawo i Sprawiedliwość,"04-01-1959, Krzeszowice",wyższe,"Społeczna Akademia Nauk w Łodzi, Wydział Zarzą...",parlamentarzysta
1,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Rafał Adamczyk,http://orka.sejm.gov.pl/Poslowie9.nsf/0/8ACA67...,13-10-2019,Sojusz Lewicy Demokratycznej,32 Katowice,12148,brak,"Koalicyjny Klub Parlamentarny Lewicy (Razem, S...","30-05-1974, Dąbrowa Górnicza",wyższe,"Politechnika Śląska, Organizacja i Zarządzanie...",samorządowiec
2,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Piotr Adamowicz,http://orka.sejm.gov.pl/Poslowie9.nsf/0/3CA579...,13-10-2019,Koalicja Obywatelska,25 Gdańsk,41795,brak,Klub Parlamentarny Koalicja Obywatelska - Plat...,"26-06-1961, Elbląg",średnie ogólne,VI LO GdaÅ„sk (1980),dziennikarz
3,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Romuald Ajchler,http://orka.sejm.gov.pl/Poslowie9.nsf/0/5E88FA...,13-10-2019,Sojusz Lewicy Demokratycznej,38 Piła,14438,"poseł II kadencji, poseł III kadencji, poseł I...","Koalicyjny Klub Parlamentarny Lewicy (Razem, S...","19-01-1949, Duszniki Wielkopolskie",średnie zawodowe,"PaÅ„stwowe Technikum Rolnicze, Rolnictwo - tec...",rolnik
4,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Adam Andruszkiewicz,http://orka.sejm.gov.pl/Poslowie9.nsf/0/82D86C...,13-10-2019,Prawo i Sprawiedliwość,24 Białystok,29829,poseł VIII kadencji,Klub Parlamentarny Prawo i Sprawiedliwość,"30-06-1990, Grajewo",wyższe,"Uniwersytet w Białymstoku, Wydział Historyczno...",parlamentarzysta


<img align="left" src="https://statici.behindthevoiceactors.com/behindthevoiceactors/_img/chars/porky-pig-animaniacs-2.3.jpg" width="300">

In [19]:
def url_to_image(url):
    try:
        # download the image, convert it to a NumPy array, and then read
        # it into OpenCV format
        time.sleep(np.random.randint(1,5))
        resp = requests.get(url)
        image = np.asarray(bytearray(resp.content), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        # return the image
        return image
    except Exception as e:
        print(f"WARNING - REQUEST ERROR @: {url}\n\t{e}")
        return

In [20]:
def encode_faces(photo_url, image_id, save_dir="images/"):
    img = url_to_image(photo_url)
    if img is None:
        print(f"WARNING - NO IMAGE FOUND @: {photo_url}")
        return np.zeros(128)
    
    # find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(img, model="cnn")
    print(f"Found: {len(face_locations)} face location(s) @ {photo_url}")

    # if no faces located return zeros
    if len(face_locations) < 1:
        return np.zeros(128)
    face_encodings = face_recognition.face_encodings(img, face_locations)

    # draw a box around faces and save processed image
    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)

    cv2.imwrite(f"{save_dir}{image_id}.png", img)

    # should be more elegant but for know is fine
    return face_encodings[0] 

In [21]:
df["face_encodings"] = [encode_faces(photo, name) for photo, name in tqdm(df[["photo_url", "name"]].values) ]

Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/8A5101B0A983511BC125849C003135B9/$File/001.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/8ACA67D7025CEF43C125849C003135BE/$File/002.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/3CA57960F0DE72EEC125849C003135C2/$File/003.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/5E88FA00DACD2F1EC125849C003135C6/$File/004.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/82D86C5B041A5CCFC125849C003135CA/$File/005.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/7817B0EB803D05C8C125849C003135CF/$File/006.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/33F9C42D15534197C125849C003135D4/$File/007.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/8DD11E7617346601C125849C003135D8/$File/008.jpg
Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/715FA42CFA5F

<img align="left" src="https://statici.behindthevoiceactors.com/behindthevoiceactors/_img/chars/porky-pig-bugs-bunnys-third-movie-1001-rabbit-tales-4.58.jpg" width="300">

In [31]:
# verifying what went wrong
df[df["face_encodings"].apply(max) == 0]

,url,name,photo_url,elected_date,list,district,votes,political_presence,club,birth,education,graduated_school,occupation,face_encodings
28,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Błąd 502 / Error 502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
480,https://www.senat.gov.pl/sklad/senatorowie/sen...,Wiesław Dobkowski,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Komitet Wyborczy Prawo i Sprawiedliwość,"okręg wyborczy nr 28, Piotrków Trybunalski",NaN,"Kadencje: X, IX, VIII, VII",Klub Parlamentarny Prawo i Sprawiedliwość,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
482,https://www.senat.gov.pl/sklad/senatorowie/sen...,Artur Dunin,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Koalicyjny Komitet Wyborczy Koalicja Obywatels...,"okręg wyborczy nr 23, Łódź",NaN,Kadencje: X,Klub Parlamentarny Koalicja Obywatelska – Plat...,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
511,https://www.senat.gov.pl/sklad/senatorowie/sen...,Małgorzata Kopiczko,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Komitet Wyborczy Prawo i Sprawiedliwość,"okręg wyborczy nr 87, Olsztyn",NaN,"Kadencje: X, IX",Klub Parlamentarny Prawo i Sprawiedliwość,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
513,https://www.senat.gov.pl/sklad/senatorowie/sen...,Krzysztof Kwiatkowski,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Komitet Wyborczy Wyborców Krzysztofa Kwiatkows...,"okręg wyborczy nr 24, Łódź",NaN,"Kadencje: X, VII",Koło Senatorów Niezależnych\nSenatorowie niezr...,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
542,https://www.senat.gov.pl/sklad/senatorowie/sen...,Krzysztof Słoń,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Komitet Wyborczy Prawo i Sprawiedliwość,"okręg wyborczy nr 83, Kielce",NaN,"Kadencje: X, IX, VIII",Klub Parlamentarny Prawo i Sprawiedliwość,NaN,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [36]:
# fixing only those with server errors, rest seems to have poor quality photos
# Joanna Borowiak
df.loc[28, df.columns[1:-1]] = [
    "Joanna Borowiak", 
    "http://orka.sejm.gov.pl/Poslowie9.nsf/0/48D129C06E4FDCE5C125849C00313631/$File/029.jpg", 
    "13-10-2019", 
    "Prawo i Sprawiedliwość", 
    "5  Toruń",
    "15200",
    "poseł VIII kadencji",
    "Klub Parlamentarny Prawo i Sprawiedliwość",
    "20-08-1967, Włocławek",
    "wyższe",
    """Wyższa Szkoła Pedagogiczna w Bydgoszczy, Wydział Pedagogiczny, Wychowanie muzyczne - magister (1993)
Uniwersytet Mikołaja Kopernika w Toruniu, Pedagogika - doktor (2005)
Kujawsko-Pomorskie Centrum Edukacji Nauczycieli we Włocławku, Organizacja i zarządzanie oświatą (2007) - studia podyplomowe
Uniwersytet im. Mikołaja Kopernika w Toruniu, Pedagogika zdolności i twórczości (2018) - studia podyplomowe
Wyższa Szkoła Kultury Społecznej i Medialnej w Toruniu, Relacje Międzynarodowe i Dyplomacja (2018) - studia podyplomowe""",
    "nauczyciel akademicki"
]

In [39]:
df.loc[28, "face_encodings"] = encode_faces(*df.loc[28, ["photo_url", "name"]])

Found: 1 face location(s) @ http://orka.sejm.gov.pl/Poslowie9.nsf/0/48D129C06E4FDCE5C125849C00313631/$File/029.jpg


In [40]:
df.to_pickle("known_faces_df.pickle")

<img align="left" src="https://i.pinimg.com/originals/0c/b4/2d/0cb42d7c7f68fed8519555dd846b307b.jpg" width="300">

In [41]:
# simple verification
import face_recognition

In [70]:
df = pd.read_pickle("known_faces_df.pickle")

In [71]:
def url_to_image(url):
    try:
        # download the image, convert it to a NumPy array, and then read
        # it into OpenCV format
        time.sleep(np.random.randint(1,5))
        resp = requests.get(url)
        image = np.asarray(bytearray(resp.content), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        # return the image
        return image
    except Exception as e:
        print(f"WARNING - REQUEST ERROR @: {url}\n\t{e}")
        return

In [72]:
sample_url = "https://gfx.wiadomosci.radiozet.pl/var/radiozetwiadomosci/storage/images/polska/polityka/schetyna-pisze-list-do-czlonkow-po.-apeluje-by-nie-oceniac-zle-wynikow-wyborow-parlamentarnych/5108515-1-pol-PL/Schetyna-pisze-list-do-czlonkow-PO-nie-oceniajmy-zle-wynikow-wyborow_article.jpg"
sample_img = url_to_image(sample_url)

In [73]:
sample_loc = face_recognition.face_locations(sample_img, model="cnn")
sample_encoding = face_recognition.face_encodings(sample_img, sample_loc)[0]

In [74]:
# compare all known faces to sample one
distance = face_recognition.face_distance(df["face_encodings"].to_list(), sample_encoding)

In [75]:
# show 10 closest
df.loc[np.argsort(distance)].head(10)

,url,name,photo_url,elected_date,list,district,votes,political_presence,club,birth,education,graduated_school,occupation,face_encodings
328,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Grzegorz Schetyna,http://orka.sejm.gov.pl/Poslowie9.nsf/0/210D0B...,13-10-2019,Koalicja Obywatelska,3 Wrocław,66859,"poseł III kadencji, poseł IV kadencji, poseł V...",Klub Parlamentarny Koalicja Obywatelska - Plat...,"18-02-1963, Opole",wyższe,"Uniwersytet Wrocławski, Wydział Filozoficzno-H...",parlamentarzysta,"[-0.04961857944726944, 0.1257019191980362, 0.0..."
323,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Jarosław Sachajko,http://orka.sejm.gov.pl/Poslowie9.nsf/0/D572D3...,13-10-2019,Polskie Stronnictwo Ludowe,7 Chełm,10651,poseł VIII kadencji,Klub Parlamentarny Koalicja Polska - Polskie S...,"08-10-1971, Lublin",wyższe,"Szkoła Główna Gospodarstwa Wiejskiego, Wydział...",ekonomista,"[-0.014987127855420113, 0.023767225444316864, ..."
476,https://www.senat.gov.pl/sklad/senatorowie/sen...,Leszek Czarnobaj,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Koalicyjny Komitet Wyborczy Koalicja Obywatels...,"okręg wyborczy nr 67, Gdańsk",NaN,"Kadencje: X, IX, VIII",Klub Parlamentarny Koalicja Obywatelska – Plat...,NaN,NaN,NaN,NaN,"[-0.07632576674222946, 0.1887936294078827, 0.0..."
364,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Wiesław Szczepański,http://orka.sejm.gov.pl/Poslowie9.nsf/0/CF67D7...,13-10-2019,Sojusz Lewicy Demokratycznej,36 Kalisz,23799,"poseł II kadencji, poseł III kadencji, poseł V...","Koalicyjny Klub Parlamentarny Lewicy (Razem, S...","01-06-1960, Chrośnica",wyższe,"Akademia Ekonomiczna w Poznaniu, Organizacja i...",ekonomista,"[-0.025104938074946404, 0.06548818200826645, 0..."
41,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Kazimierz Choma,http://orka.sejm.gov.pl/Poslowie9.nsf/0/9A6E7F...,13-10-2019,Prawo i Sprawiedliwość,6 Lublin,13777,brak,Klub Parlamentarny Prawo i Sprawiedliwość,"16-12-1957, Kraśnik",wyższe,"Akademia Górniczo-Hutnicza w Krakowie, Wydział...",urzędnik państwowy,"[-0.07091505825519562, 0.07257754355669022, 0...."
59,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Eugeniusz Czykwin,http://orka.sejm.gov.pl/Poslowie9.nsf/0/B4AB33...,13-10-2019,Koalicja Obywatelska,24 Białystok,14083,"poseł IX kadencji, poseł X kadencji, poseł I k...",Klub Parlamentarny Koalicja Obywatelska - Plat...,"12-09-1949, Orla",wyższe,"Politechnika Warszawska, Wydział Transportu, t...",dziennikarz,"[-0.0974792018532753, 0.02889080159366131, -0...."
465,https://www.senat.gov.pl/sklad/senatorowie/sen...,Ryszard Bober,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,NaN,"okręg wyborczy nr 12, Toruń",NaN,Komitet Wyborczy Polskie Stronnictwo Ludowe,Klub Parlamentarny Koalicja Polska – Polskie S...,NaN,NaN,NaN,NaN,"[-0.14611530303955078, 0.1410762220621109, 0.0..."
115,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Cezary Grabarczyk,http://orka.sejm.gov.pl/Poslowie9.nsf/0/B1AA3C...,13-10-2019,Koalicja Obywatelska,10 Piotrków Trybunalski,16357,"poseł IV kadencji, poseł V kadencji, poseł VI ...",Klub Parlamentarny Koalicja Obywatelska - Plat...,"26-04-1960, Łódź",wyższe,"Uniwersytet Łódzki, Wydział Prawa i Administra...",adwokat,"[-0.07574651390314102, 0.050695519894361496, 0..."
439,https://www.sejm.gov.pl/Sejm9.nsf/posel.xsp?id...,Paweł Zalewski,http://orka.sejm.gov.pl/Poslowie9.nsf/0/E67635...,13-10-2019,Koalicja Obywatelska,20 Warszawa,12248,"poseł I kadencji, poseł V kadencji, poseł VI k...",Klub Parlamentarny Koalicja Obywatelska - Plat...,"25-09-1964, Warszawa",wyższe,"Uniwersytet Warszawski, Wydział Historyczny, h...",konsultant,"[-0.052102334797382355, 0.13092994689941406, 0..."
533,https://www.senat.gov.pl/sklad/senatorowie/sen...,Aleksander Pociej,https://www.senat.gov.pl/gfx/senat/_thumbs/pl/...,NaN,Koalicyjny Komitet Wyborczy Koalicja Obywatels...,"okręg wyborczy nr 45, Warszawa",NaN,"Kadencje: X, IX, VIII",Klub Parlamentarny Koalicja Obywatelska – Plat...,NaN,NaN,NaN,NaN,"[-0.15876087546348572, 0.1